In [1]:
import os
%pwd

'/Users/ofotech_fitri/Documents/fitri_github/data-science-with-mlflow/research'

In [2]:
os.chdir("../")
%pwd

'/Users/ofotech_fitri/Documents/fitri_github/data-science-with-mlflow'

# Update Entity

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerElasticNetConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    model_output: str
    alpha: float
    l1_ratio: float
    target_column: str

@dataclass(frozen=True)
class ModelTrainerRandomForestRegressionConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    model_output: str
    n_estimators: int
    random_state: int
    target_column: str

@dataclass(frozen=True)
class ModelTrainerKNNRegressionConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    model_output: str
    n_neighbors: int
    target_column: str

@dataclass(frozen=True)
class ModelTrainerXGboostRegressionConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    model_output: str
    objective: str
    learning_rate: float
    n_estimators: int
    max_depth: int
    min_child_weight: int
    subsample: float
    colsample_bytree: float
    colsample_bylevel: float
    gamma: int
    reg_alpha: int
    reg_lambda: int
    scale_pos_weight: int
    target_column: str

# Update the configuration manager in src config

In [4]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories
from mlProject import logger

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    # def get_model_trainer_config(self) -> ModelTrainerConfig:
    def get_model_trainer_config(self):
        config = self.config.model_trainer
        model_name = config.model_name
        
        def get_model_params(model_name: str):
            if model_name in self.params:
                return self.params[model_name]
            else:
                logger.error("Model name not exist in params.yaml")
                return None  # Handle the case where model_name is not found in params
        
        params = get_model_params(model_name)
        # print(params)
        # params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        if model_name == "ElasticNet":
            model_trainer_config = ModelTrainerElasticNetConfig(
                root_dir=config.root_dir,
                train_data_path = config.train_data_path,
                test_data_path = config.test_data_path,
                model_name = config.model_name,
                model_output = config.model_output,
                alpha = params.alpha,
                l1_ratio = params.l1_ratio,
                target_column = schema.name
            )
        elif model_name == "RandomForestRegression":
            model_trainer_config = ModelTrainerRandomForestRegressionConfig(
                root_dir=config.root_dir,
                train_data_path = config.train_data_path,
                test_data_path = config.test_data_path,
                model_name = config.model_name,
                model_output = config.model_output,
                n_estimators = params.n_estimators,
                random_state = params.random_state,
                target_column = schema.name
            )

        elif model_name == "KNNRegression":
            model_trainer_config = ModelTrainerKNNRegressionConfig(
                root_dir=config.root_dir,
                train_data_path = config.train_data_path,
                test_data_path = config.test_data_path,
                model_name = config.model_name,
                model_output = config.model_output,
                n_neighbors = params.n_neighbors,
                target_column = schema.name
            )

        elif model_name == "XGboostRegression":
            model_trainer_config = ModelTrainerXGboostRegressionConfig(
                root_dir=config.root_dir,
                train_data_path = config.train_data_path,
                test_data_path = config.test_data_path,
                model_name = config.model_name,
                model_output = config.model_output,
                objective = params.objective,
                learning_rate = params.learning_rate,
                n_estimators = params.n_estimators,
                max_depth = params.max_depth,
                min_child_weight = params.min_child_weight,
                subsample = params.subsample,
                colsample_bytree = params.colsample_bytree,
                colsample_bylevel = params.colsample_bylevel,
                gamma = params.gamma,
                reg_alpha = params.reg_alpha,
                reg_lambda = params.reg_lambda,
                scale_pos_weight = params.scale_pos_weight,
                target_column = schema.name
            )

        return model_trainer_config

# Update the components

In [5]:
import pandas as pd
import os
from xgboost import XGBRegressor
from mlProject import logger
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

import joblib

In [6]:
class ModelTrainer:
    # def __init__(self, config: ModelTrainerConfig):
    def __init__(self, config):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        model_name = self.config.model_name

        # add if using different pretrain model
        if model_name == "ElasticNet":
            lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
            lr.fit(train_x, train_y)

            joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_output))
        
        elif model_name == "RandomForestRegression":
            regressor = RandomForestRegressor(n_estimators=self.config.n_estimators, random_state=self.config.random_state)
            regressor.fit(train_x, train_y)

            joblib.dump(regressor, os.path.join(self.config.root_dir, self.config.model_output))

        elif model_name == "KNNRegression":
            knn_regressor = KNeighborsRegressor(n_neighbors=self.config.n_neighbors)
            knn_regressor.fit(train_x, train_y)

            joblib.dump(knn_regressor, os.path.join(self.config.root_dir, self.config.model_output))
        
        elif model_name == "XGboostRegression":
            xgb_regressor = XGBRegressor(objective=self.config.objective, learning_rate=self.config.learning_rate, n_estimators=self.config.n_estimators, max_depth=self.config.max_depth, min_child_weight=self.config.min_child_weight, subsample=self.config.subsample, colsample_bytree=self.config.colsample_bytree, colsample_bylevel=self.config.colsample_bylevel, gamma=self.config.gamma, reg_alpha=self.config.reg_alpha, reg_lambda=self.config.reg_lambda, scale_pos_weight=self.config.scale_pos_weight)
            xgb_regressor.fit(train_x, train_y)

            joblib.dump(xgb_regressor, os.path.join(self.config.root_dir, self.config.model_output))


# Update the pipeline

In [7]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-12-15 11:28:22,557: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-15 11:28:22,559: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-15 11:28:22,561: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-15 11:28:22,562: INFO: common: created directory at: artifacts]
[2023-12-15 11:28:22,563: INFO: common: created directory at: artifacts/04_model_trainer]
